# Make word graphs from csv

From a csv file with at least three columns, two with words and one with a frequency. 

In [1]:
import sqlite3
import pandas as pd

# Functions 

In [2]:
def query(db, sql, param=()):
    """ Query a sqlitedatabase with sql and param"""
    with sqlite3.connect(db) as con:
        cur = con.cursor()
        cur.execute(sql, param)
    return cur.fetchall()

In [3]:
def create_database_from_df(db, df):
    """
        db is the name of an empty database, df is a dataframe with columns named first, second, freq
        A table named ngram is created with two secondary tables
    """
    print('creating database from dataframe')
    with sqlite3.connect(db) as con:
        df[['first','second','freq']].to_sql('ngram', con)
    print('indexing main table')
    query(db, "create index '_ft_' on ngram (first, second)")
    query(db, "create index '_tf_' on ngram (second, first)")
    query(db, "create table firstfreq (first varchar, freq int)")
    query(db, "create table secondfreq (second varchar, freq int)")
    query(db, "insert into firstfreq select first, sum(freq) from ngram group by first")
    query(db, "insert into secondfreq select second, sum(freq) from ngram group by second")
    print('indexing secondary tables')
    query(db, "create index _ff_ on firstfreq (first, freq)")
    query(db, "create index _ftf_ on secondfreq (second, freq)")

In [4]:
def make_word_graph(db):
    print('create graph of word pairs')
    query(db, "create table word_graph (first varchar, second varchar, freq int, pmi float)")
    query(db, "insert into word_graph select  a.first, a.second, sum(a.freq) as f, pow(a.freq,2)*1.0/ (f.freq * t.freq) as pmi from ngram as a, firstfreq as f, secondfreq as t where a.first = f.first and a.second = t.second group by a.first, a.second")
    print('indexing graph')
    query(db, "create index _gftp_ on word_graph (first, pmi)")
    query(db, "create index _gtfp_ on word_graph (second, pmi)")

In [5]:
def check_graph_first(x, top = 20):
    return query(word_pair_database, "select * from word_graph where first = ? order by pmi desc limit ?", (x, top))
def check_graph_second(x, top = 20):
    return query(word_pair_database, "select * from word_graph where second = ? order by pmi desc limit ?", (x, top))

## File to store word graph - change name as appropriate

In [15]:
word_pair_database = "book_news_word_pairs.db"

## CSV-file structure


Specify name of file, and a mapping from the columns to "first", "second" and "freq". The graph is built as weighted graph with edges  `(first, second)`, where frequency and compututations based on frequencies are weights. 

If the CSV file has for example five columns, and the `first` element is in 2nd position with the `second` in 4th, and frequency is in 5th, just indicate the mapping with a schematic description of the columns so that `first`, `second` and `freq` are matched up with the appropriate columns, for example like this: 

```
['p0', 'first', 'p2', 'second', 'freq']
```
The names for unused columns are arbitrary. Only the words in the `first` and `second` columns with the weights in `freq` are used in constructing the graph. 

The code will aggregate the columns, summing up the extraneous columns. 

$$\textrm{columns} = \sum_{x,y \in \textrm{p0}, \textrm{p2} } {\textrm{columns}(\textrm{freq})_{[\textrm{p0}, \textrm{first}, \textrm{p2}, \textrm{second} ]}}$$

The aggregation produces a table with aggregated frequencies with only three columns:

```
['first', 'second', 'freq']
```


### Specify CSV data  

csv_data = {
    'file':"../../ngram_2021_coordination/coord-bok.csv",
    'columns': ['lang', 'first', 'coord', 'second','freq']
}

In [16]:
csv_data = {
    'file':"avis_bøker_tre_kolonner.csv",
    'columns': [ 'first', 'coord', 'second','freq']
}

# Read CSV and aggregate the frequencies

If csv has a header, set `header = 0`, and if there is a separate (initial) index column, set `index_col = 0`. If there are errors, just add cells and inspect the variable `data`.

In [18]:
data = pd.read_csv(csv_data['file'], header = 0, index_col = 0)

In [19]:
data.columns = csv_data['columns']

In [20]:
if len(data.columns) > 3:
    data = data.groupby(['first', 'second']).sum('freq').reset_index()

# Create basic word pairs

Store the dataframe in an sqlite-database with frequencies together with derived frequencies - table is called `ngram`. Words with frequencies from the first and second columns are extracted and added up into separate tables dubbed `firstfreq` and `secondfreq`.

In [21]:
create_database_from_df(word_pair_database, data)

creating database from dataframe
indexing main table
indexing secondary tables


# Make table of word pairs

The table of word pairs is computed using the ngram-table and named `word_graph`. Indexed and ready to use.

The columns of the `word_graph` table are `first, second, freq, pmi` where freq contains the original frequencies, and pmi is the computed association value, using the following formula:

$$\frac{\textrm{freq}(x,y)^2}{\textrm{freq}(x)*\textrm{freq}(y)}$$

The frequency is squared to give the cooccurence frequency a higher weight. The marginals `freq(x)` are aggregated from the word positions. For each word $w$ it is associated with two frequencies, one frequency from the occurence in the first column, and one from the second column.

The actual computation can be changed in the definition of the function `make_word_graph`. Locate the part of the expression `pow(a.freq,2)*1.0/ (f.freq * t.freq)` in one of the sql-queries, and substitute it with a preferred computation.

In [22]:
make_word_graph(word_pair_database)

create graph of word pairs
indexing graph


# Querying word graphs

In [23]:
check_graph_first('Ibsen')

[('Ibsen', 'Bjørnson', 14960, 0.11225164442292179),
 ('Ibsen', 'Hamsun', 3208, 0.016603137431383096),
 ('Ibsen', 'Strindberg', 1993, 0.009848228350741436),
 ('Ibsen', 'Bjornson', 506, 0.004713533750995571),
 ('Ibsen', 'Bjørnstjerne', 1874, 0.003941420429056921),
 ('Ibsen', 'Shakespeare', 857, 0.0025556298516234326),
 ('Ibsen', 'Kielland', 842, 0.0018248193312675124),
 ('Ibsen', 'moderniteten', 196, 0.001678853754033853),
 ('Ibsen', 'Grieg', 1327, 0.0012141432335555356),
 ('Ibsen', 'Bjsrnson', 134, 0.0010847490065025737),
 ('Ibsen', 'Brandes', 280, 0.0009993177107344364),
 ('Ibsen', 'skandinavismen', 159, 0.0009693031013213325),
 ('Ibsen', 'Bjørnsons', 632, 0.0008654149644214488),
 ('Ibsen', 'Munch', 643, 0.000762903528136489),
 ('Ibsen', 'Gynt', 50, 0.0007188969245589567),
 ('Ibsen', 'Fosse', 383, 0.0006887934093182809),
 ('Ibsen', 'Snoilsky', 41, 0.0005894954781383446),
 ('Ibsen', 'Sigval', 163, 0.0005269068245132944),
 ('Ibsen', 'Holberg', 339, 0.0005000989919324447),
 ('Ibsen', 'Wen

In [24]:
check_graph_second('Ibsen')

[('Bjørnson', 'Ibsen', 10176, 0.07892269095685274),
 ('Shakespeare', 'Ibsen', 1105, 0.006000911962673814),
 ('Bjornson', 'Ibsen', 420, 0.005280769914699401),
 ('Bjsrnson', 'Ibsen', 167, 0.0021529822459443893),
 ('Strindberg', 'Ibsen', 477, 0.0016279760976311995),
 ('Hamsun', 'Ibsen', 861, 0.0014431773344294396),
 ('Holberg', 'Ibsen', 535, 0.0010180652019854165),
 ('Kierkegaard', 'Ibsen', 254, 0.0007111177343169088),
 ('Munch', 'Ibsen', 689, 0.000604182678432694),
 ('Wergeland', 'Ibsen', 558, 0.00044568896794842396),
 ('Bjprnson', 'Ibsen', 10, 0.0004068348250610252),
 ('Vjsrnson', 'Ibsen', 20, 0.00037845100005676766),
 ('Grimstadtid', 'Ibsen', 7, 0.00028478437754271763),
 ('Tsjekhov', 'Ibsen', 40, 0.0002805757414213967),
 ('Botten-Hansen', 'Ibsen', 33, 0.00026529528412662064),
 ('Vjornson', 'Ibsen', 15, 0.0002473995557803532),
 ('Bjørnsom', 'Ibsen', 6, 0.00024410089503661513),
 ('Bergsøe', 'Ibsen', 6, 0.00024410089503661513),
 ('digtning', 'Ibsen', 77, 0.0002328304708288435),
 ('Bjomson